<a href="https://colab.research.google.com/github/snoop2head/OIA_Text_Wrangling/blob/master/_Department_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import pandas as pd
from pandas.api.types import CategoricalDtype # 그래프의 값을 정렬해서 보기위해
import numpy as np
print(pd.__version__)
print(np.__version__)

0.25.3
1.17.4


In [59]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [69]:
file_name = "_department_t.csv"
path = "/content/gdrive/My Drive/_OIA_Project/" + file_name
df = pd.read_csv(path)
# 데이터의 크기가 어느정도인지 본다.
df.shape

(141, 182)

In [70]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181
0,Australian National University,Bond University,Curtin University of Technology,La Trobe University,Monash University,University of New South Wales,University of Queensland,University of Sydney,"University of Technology, Sydney",FH Kufstein Tirol University of Applied Sciences,University of Vienna,Vienna University of Economics and Business,Universite Catholique De Louvain,Brock University,McGill University,Simon Fraser University,University of Alberta,University of British Columbia,University of Manitoba,University of Toronto,York University: Schulich School of Business,Fudan University,Wuhan University,Zhejiang Normal University,Charles University,University of Copenhagen,Aalto University,University of Eastern Finland,University of Helsinki,"University of Jyvaskyla, School of Business & ...",Ecole Superieure du Commerce Exterieur (ESCE),ESC (Business School) Amiens,ESCEM (France Business School) Tours-Poitiers,ISC PARIS - School of Management,KEDGE Business School,Pole ESG-Paris Graduate School of Management,Sciences-Po,Eberhard Karls Universitat Tubingen,European Business School,Freie Universitat Berlin,...,SUNY at Stony Brook,Syracuse University,Temple University,Towson University,University of Akron,University of Alabama,University of California Berkeley,University of California Davis,University of California Irvine,University of California Los Angeles,University of California Riverside,University of California San Diego,University of California Santa Barbara,University of California Santa Cruz,University of Connecticut,University of Georgia,University of Hawaii at Manoa,University of Illinois at Urbana-Champaign,University of Maryland at College Park,University of Massachusetts Amherst,University of Michigan,University of Minnesota Morris,University of North Carolina at Chapel Hill,University of North Carolina at Greensboro,University of Oregon,University of Pennsylvania,University of Pittsburgh,University of Richmond,University of Scranton,University of South Florida,University of Tennessee Knoxville,University of Texas at Arlington,University of Utah,University of Washington,University of Wisconsin-Madison,Washington College,Washington University in St. Louis,Western Kentucky University,Western Washington University,Westminster College
1,UD IS,경영학,경영학,경영학,UIC,GSIS,InformationandInteractionDesign,UD PSIR,UIC TAD,경영학,경영학,경영학,경영학,감호학,건축공학,EIC,"InternationalTrade,FinanceandManagement",UIC,건축공학,Psychology,경영학원,GSIS,경영학,GSIS,건축공학,UD CLC,경영학,경영학,경제학,경영학,UIC,경영학,경영학,UIC ASD,EIC,경영학,GSIS,경영학,경영학,철학,...,UD Economics,UD Economics,CTM,경영학,건축학,건축학,UIC ASD,EIC,UD Economics,UIC,SchoolofBusiness,BioConvergence,UD CLC,UD Economics,경영학,UD Economics,UD CLC,EESE,UD PSIR,건축공학,경영학,간호학,경영학,경영학,경영학,UIC,경영학,경영학,UIC ASD,경영학,경영학,간호학,EESE,UD Economics,UIC,경영학,UD Economics,UD Economics,UD Economics,경영학
2,경영학,경제학,경제학,경영학,UIC,건축학,건축공학,UD CLC,건축공학,경영학,경제학,경영학,경영학,경영학,경영학,경영학,건축학,경영학,건축학,UIC,경영학,경영학,경영학,경영학,경영학,UD Economics,경영학,공대학세라믹공학,경제학,경영학,UIC,경영학,경영학,UD Economics,경영학,경영학,UD CLC,경제학,경영학,경영학,...,건축공학,건축학,UIC CTM,경영학,경영학,경영학,EIC,TAD,UD Economics,UnderwoodInternational,UIC,UD IS,EastAsiaInternationalCollege,Politics,경영학,경영학,UIC,건축공학,UD PSIR,경영학,경영학,경영학/경제,경영학,경영학,경영학,UD Economics,경영학,경영학,경영학,경영학,경영학,건축공학,경영학,UIC LSBT,UIC ASD,경영학,UIC ASD,건축공학,경영학,경영학
3,경제학,경제학,국어국문,교육학,경영학,경영학,경영학,UIC ASD,건축학,경영학,경제학,경영학,경영학,경영학,경영학,경영학,경영학,경영학,경영학,UIC,경영학,경영학,경영학,경영학,경영학,UnderwoodDivision,경영학,교육학,교육학,경영학,UD Economics,경영학,경영학,경영학,경영학,경영학,UD Economics,독어독문,경영학,교육학,...,경영학,경영학,경영학,경영학,경영학,경영학,UD,UIC,UIC QRM,건축학,UD IS,Techno-ArtDivision,UD CLC,경영학,경영학,경영학,UIC ASD,건축학,경영학,경영학,경영학,경영학,경영학,경영학,경영학,UD Economics,경영학,경영학,경영학,경영학,경영학,경영학,경영학,UD Economics,경영학,경영학,경영학,경영학,경영학,경영학
4,경제학,국어국문,사회학,교육학,경영학,경영학,경

In [0]:
df.columns = df.iloc[0]

In [0]:
header_list = df.columns.to_list()

In [74]:
df = df[1:]
df

,Australian National University,Bond University,Curtin University of Technology,La Trobe University,Monash University,University of New South Wales,University of Queensland,University of Sydney,"University of Technology, Sydney",FH Kufstein Tirol University of Applied Sciences,University of Vienna,Vienna University of Economics and Business,Universite Catholique De Louvain,Brock University,McGill University,Simon Fraser University,University of Alberta,University of British Columbia,University of Manitoba,University of Toronto,York University: Schulich School of Business,Fudan University,Wuhan University,Zhejiang Normal University,Charles University,University of Copenhagen,Aalto University,University of Eastern Finland,University of Helsinki,"University of Jyvaskyla, School of Business & Economics",Ecole Superieure du Commerce Exterieur (ESCE),ESC (Business School) Amiens,ESCEM (France Business School) Tours-Poitiers,ISC PARIS - School of Management,KEDGE Business School,Pole ESG-Paris Graduate School of Management,Sciences-Po,Eberhard Karls Universitat Tubingen,European Business School,Freie Universitat Berlin,...,SUNY at Stony Brook,Syracuse University,Temple University,Towson University,University of Akron,University of Alabama,University of California Berkeley,University of California Davis,University of California Irvine,University of California Los Angeles,University of California Riverside,University of California San Diego,University of California Santa Barbara,University of California Santa Cruz,University of Connecticut,University of Georgia,University of Hawaii at Manoa,University of Illinois at Urbana-Champaign,University of Maryland at College Park,University of Massachusetts Amherst,University of Michigan,University of Minnesota Morris,University of North Carolina at Chapel Hill,University of North Carolina at Greensboro,University of Oregon,University of Pennsylvania,University of Pittsburgh,University of Richmond,University of Scranton,University of South Florida,University of Tennessee Knoxville,University of Texas at Arlington,University of Utah,University of Washington,University of Wisconsin-Madison,Washington College,Washington University in St. Louis,Western Kentucky University,Western Washington University,Westminster College
1,UD IS,경영학,경영학,경영학,UIC,GSIS,InformationandInteractionDesign,UD PSIR,UIC TAD,경영학,경영학,경영학,경영학,감호학,건축공학,EIC,"InternationalTrade,FinanceandManagement",UIC,건축공학,Psychology,경영학원,GSIS,경영학,GSIS,건축공학,UD CLC,경영학,경영학,경제학,경영학,UIC,경영학,경영학,UIC ASD,EIC,경영학,GSIS,경영학,경영학,철학,...,UD Economics,UD Economics,CTM,경영학,건축학,건축학,UIC ASD,EIC,UD Economics,UIC,SchoolofBusiness,BioConvergence,UD CLC,UD Economics,경영학,UD Economics,UD CLC,EESE,UD PSIR,건축공학,경영학,간호학,경영학,경영학,경영학,UIC,경영학,경영학,UIC ASD,경영학,경영학,간호학,EESE,UD Economics,UIC,경영학,UD Economics,UD Economics,UD Economics,경영학
2,경영학,경제학,경제학,경영학,UIC,건축학,건축공학,UD CLC,건축공학,경영학,경제학,경영학,경영학,경영학,경영학,경영학,건축학,경영학,건축학,UIC,경영학,경영학,경영학,경영학,경영학,UD Economics,경영학,공대학세라믹공학,경제학,경영학,UIC,경영학,경영학,UD Economics,경영학,경영학,UD CLC,경제학,경영학,경영학,...,건축공학,건축학,UIC CTM,경영학,경영학,경영학,EIC,TAD,UD Economics,UnderwoodInternational,UIC,UD IS,EastAsiaInternationalCollege,Politics,경영학,경영학,UIC,건축공학,UD PSIR,경영학,경영학,경영학/경제,경영학,경영학,경영학,UD Economics,경영학,경영학,경영학,경영학,경영학,건축공학,경영학,UIC LSBT,UIC ASD,경영학,UIC ASD,건축공학,경영학,경영학
3,경제학,경제학,국어국문,교육학,경영학,경영학,경영학,UIC ASD,건축학,경영학,경제학,경영학,경영학,경영학,경영학,경영학,경영학,경영학,경영학,UIC,경영학,경영학,경영학,경영학,경영학,UnderwoodDivision,경영학,교육학,교육학,경영학,UD Economics,경영학,경영학,경영학,경영학,경영학,UD Economics,독어독문,경영학,교육학,...,경영학,경영학,경영학,경영학,경영학,경영학,UD,UIC,UIC QRM,건축학,UD IS,Techno-ArtDivision,UD CLC,경영학,경영학,경영학,UIC ASD,건축학,경영학,경영학,경영학,경영학,경영학,경영학,경영학,UD Economics,경영학,경영학,경영학,경영학,경영학,경영학,경영학,UD Economics,경영학,경영학,경영학,경영학,경영학,경영학
4,경제학,국어국문,사회학,교육학,경영학,경영학,경영학,UIC,건축학,경영학,경제학,경영학,경영학,경영학,경영학,경영학,경영학,경영학,경영학,UIC,경영학,경영학,경영학,교육학,경영학,건축학,경영학,교육학,나노학공학,경영학,UD IS,경영학,경영학,경영학,경영학,경제학,NaN,독어독문,경영학,독어독문,...,경영학,경영학,경영학,경영학,경영학,경제학,UD 경제,UIC,경영학,경영학,경영학,UD Economics,UD IS,경영학,경영학,경영학,UD IS,건축학,경영학,경영학,경영학,경영학,경영학,경영학,경영학,UD CLC,경제학

In [0]:
import numpy as np

def single_dp_dict(df_column):
  # university_name = df_column[0]
  single_column = df_column[0:]
  single_column_list = single_column.to_list()
  column_lst_without_nan = [x for x in single_column_list if x == x]

  splitted_list = []
  for i in column_lst_without_nan:
    if "/" in i:
      # print(i)
      double_element = i.split("/")
      # print(double_element)
      # splitted_list.remove(i)
      splitted_list += double_element
    elif "," in i:
      double_element = i.split(",")
      splitted_list += double_element
    else:
      splitted_list.append(i)
      pass
  splitted_list

  from collections import defaultdict
  fq= defaultdict( int )
  for w in splitted_list:
      fq[w] += 1
  number_of_departments = len(splitted_list)
  # print(number_of_departments)
  # print(university_name)
  dictionary = dict(fq)
  return dictionary

In [93]:
from statistics import variance

# df_column = df[header_list[25]]
df_column = df['Aalto University']
single_dict = single_dp_dict(df_column)
print(single_dict)

variance(single_dict[k] for k in single_dict)

{'경영학': 39, '경제학': 11, '교육학': 1, '국어국문': 2, '도시공학': 1, '문헌정보학': 1, '사회학': 2, '상경계열': 3, '신문방송학': 2, '영어영문': 2, '응용통계학': 3, '정치외교학': 2, '철학': 2, '산업공학': 1}


101.36263736263734

In [0]:
list_of_dict = []
for i in header_list:
  df_column = df[i]
  single_dict = single_dp_dict(df_column)
  list_of_dict.append(single_dict)
department_matrix = pd.DataFrame(list_of_dict)
department_matrix.fillna(0)

department_matrix.to_csv("/content/gdrive/My Drive/_OIA_Project/_department_matrix_mark4.csv",index=False,encoding="utf-8")

In [0]:
department_matrix.plot.hist()

In [0]:
p = r'.*(UD|Econ|UIC).*'
finance = df[df['title'].str.match(p) |
           df['content'].str.match(p, flags=re.MULTILINE)]
finance.shape

In [0]:
from math import log10

# =======================================
# -- TF-IDF function
# =======================================
def f(t, d):
    # d is document == tokens
    return d.count(t)

def tf(t, d):
    # d is document == tokens
    return 0.5 + 0.5*f(t,d)/max([f(w,d) for w in d])

def idf(t, D):
    # D is documents == document list
    numerator = len(D)
    denominator = 1 + len([ True for d in D if t in d])
    return log10(numerator/denominator)

def tfidf(t, d, D):
    return tf(t,d)*idf(t, D)

def tokenizer(d):
    # return [ t for t in d.split() if len(t) > 1 ]
    return d.split()

def tfidfScorer(D):
    tokenized_D = [tokenizer(d) for d in D]
    result = []
    for d in tokenized_D:
        result.append([(t, tfidf(t, d, tokenized_D)) for t in d])
    return result


if __name__ == '__main__':
    corpus = ['동해물과 백두산이 마르고 닳도록 하느님이 보우하사 우리나라 만세 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세',
              '남산 위에 저 소나무 철갑을 두른 듯 바람서리 불변함은 우리 기상일세 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세',
              '가을 하늘 공활한데 높고 구름 없이 밝은 달은 우리 가슴 일편단심일세 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세',
              '이 기상과 이 맘으로 충성을 다하여 괴로우나 즐거우나 나라 사랑하세 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세',
              '오 필승 코리아 오 필승 코리아 오 필승 코리아 오 오레 오레 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세']

    for i, doc in enumerate(tfidfScorer(corpus)):
        print('====== document[%d] ======' % i)
        print(doc)

        # http://blog.naver.com/PostView.nhn?blogId=vangarang&logNo=221072014624&parentCategoryNo=&categoryNo=35&viewDate=&isShowPopularPosts=true&from=search

====== document[0] ======
[('동해물과', 0.3979400086720376), ('백두산이', 0.3979400086720376), ('마르고', 0.3979400086720376), ('닳도록', 0.3979400086720376), ('하느님이', 0.3979400086720376), ('보우하사', 0.3979400086720376), ('우리나라', 0.3979400086720376), ('만세', 0.3979400086720376), ('무궁화', -0.07918124604762482), ('삼천리', -0.07918124604762482), ('화려', -0.07918124604762482), ('강산', -0.07918124604762482), ('대한사람', -0.07918124604762482), ('대한으로', -0.07918124604762482), ('길이', -0.07918124604762482), ('보전하세', -0.07918124604762482)]
====== document[1] ======
[('남산', 0.3979400086720376), ('위에', 0.3979400086720376), ('저', 0.3979400086720376), ('소나무', 0.3979400086720376), ('철갑을', 0.3979400086720376), ('두른', 0.3979400086720376), ('듯', 0.3979400086720376), ('바람서리', 0.3979400086720376), ('불변함은', 0.3979400086720376), ('우리', 0.22184874961635637), ('기상일세', 0.3979400086720376), ('무궁화', -0.07918124604762482), ('삼천리', -0.07918124604762482), ('화려', -0.07918124604762482), ('강산', -0.07918124604762482), ('대한사람', -0.0791812460476

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

documents = ["This little kitty came to play when I was eating at a restaurant.",
             "Merley has the best squooshy kitten belly.",
             "Google Translate app is incredible.",
             "If you open 100 tab in google you get a smiley face.",
             "Best cat photo I've ever taken.",
             "Climbing ninja cat.",
             "Impressed with google map feedback.",
             "Key promoter extension for Google Chrome."]

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")
print("Prediction")

Y = vectorizer.transform(["chrome browser to open."])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["My cat is hungry."])
prediction = model.predict(Y)
print(prediction)